In [2]:
from qreduce.cs_vqe_model import cs_vqe_model
from qreduce.utils import *
from itertools import combinations
import json
import numpy as np

In [3]:
with open('data/model_data.json', 'r') as infile:
        mol_data = json.load(infile)
mol_data.keys()

dict_keys(['H2O_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'CH2_STO-3G_SINGLET', 'BeH2_STO-3G_SINGLET', 'HeH+_6-311G_SINGLET', 'BeH+_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'OH+_STO-3G_SINGLET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'NH_STO-3G_SINGLET', 'BH_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'CH2_STO-3G_TRIPLET', 'C_STO-3G_TRIPLET'])

In [32]:
mol=mol_data['C_STO-3G_TRIPLET']
ham=mol['ham']
n_q=mol['num_qubits']
set_nc = mol['terms_noncon']
cs_vqe_mol = cs_vqe_model(ham, noncontextual_set = set_nc, single_pauli='X')

In [33]:
print("Generators:", cs_vqe_mol.generators)
print("Clique reps:", cs_vqe_mol.cliquereps)
#match=mol['X_index']==cs_vqe_mol.anti_clique_qubit
#print('Anti-clique qubit match?', match, '\n')

print("G assignment:", cs_vqe_mol.nu)#, ' | original:', np.array(list(mol['G'].values()), dtype=int))
print("r vector:", cs_vqe_mol.r)
print("Noncon nrg:", cs_vqe_mol.ngs_energy)
print('NGS matches original?', abs(cs_vqe_mol.ngs_energy-mol['noncon'])<1e-13, '\n')

Generators: ['ZZZZZ', 'ZZZIZ', 'ZIZIZ', 'ZZIIZ', 'ZZZII']
Clique reps: ['ZZZZZ', 'ZZZXZ']
G assignment: [ 1 -1 -1  1  1]
r vector: [1.00000000e+00 1.59198708e-08]
Noncon nrg: -37.19839256373158
NGS matches original? True 



In [34]:
num_sim_q = mol['chem_acc_num_q']
exact = exact_gs_energy(cs_vqe_mol.hamiltonian)
print('Exact energy:',exact)
print('Noncon error:', cs_vqe_mol.ngs_energy-exact)
print(f'Target_error for {num_sim_q} qubits:', exact_gs_energy(mol['ham_reduced'][num_sim_q])-exact)

Exact energy: -37.21873355063646
Noncon error: 0.020340986904876956
Target_error for 2 qubits: 0.00011593095587159041


In [35]:
optimal_error = {}
for num_sim_q in range(1, cs_vqe_mol.num_qubits):
    cs_vqe_errors = []
    for order in combinations(range(len(cs_vqe_mol.generators)), cs_vqe_mol.num_qubits - num_sim_q):
        order = list(order)
        ham_cs = cs_vqe_mol.contextual_subspace_hamiltonian(order)
        cs_energy = exact_gs_energy(ham_cs)
        cs_vqe_errors.append((cs_energy-exact, order))
    cs_vqe_errors = sorted(cs_vqe_errors, key=lambda x:x[0])
    optimal_error[num_sim_q] = cs_vqe_errors[0]
    
optimal_error

{1: (0.020340986904884062, [0, 1, 3, 4]),
 2: (0.020261906572379473, [0, 1, 3]),
 3: (8.187161783723695e-05, [1, 3]),
 4: (4.117318268725967e-05, [1])}

In [43]:
ham_cs = cs_vqe_mol.contextual_subspace_hamiltonian([2,0,1])
print(number_of_qubits(ham_cs))
cs_error = exact_gs_energy(ham_cs)-exact
cs_error

2


0.02034098690485564